In [1]:
import numpy
print("numpy:   " + numpy.version.version)

import sklearn
import sklearn.metrics
print("sklearn: " + sklearn.__version__)

numpy:   1.19.2
sklearn: 0.23.2


In [2]:
# IDs for loading data below
embeddingsDir = "/home/eml4u/EML4U/data/interim-results/"

idAmericanA = "20100408-american-films"
idAmericanB = "20201101-american-films"
idBritishA  = "20100408-british-films"
idBritishB  = "20201101-british-films"
idIndianA   = "20100408-indian-films"
idIndianB   = "20201101-indian-films"

In [3]:
# Load text files containing embeddings
def getTextfile(dir, id, printfile = True):
    file = dir + id + ".txt"
    if printfile:
        print(file)
    return file

fileEmbeddingsAmericanA = getTextfile(embeddingsDir, idAmericanA)
fileEmbeddingsAmericanB = getTextfile(embeddingsDir, idAmericanB)
fileEmbeddingsBritishA  = getTextfile(embeddingsDir, idBritishA)
fileEmbeddingsBritishB  = getTextfile(embeddingsDir, idBritishB)
fileEmbeddingsIndianA   = getTextfile(embeddingsDir, idIndianA)
fileEmbeddingsIndianB   = getTextfile(embeddingsDir, idIndianB)

/home/eml4u/EML4U/data/interim-results/20100408-american-films.txt
/home/eml4u/EML4U/data/interim-results/20201101-american-films.txt
/home/eml4u/EML4U/data/interim-results/20100408-british-films.txt
/home/eml4u/EML4U/data/interim-results/20201101-british-films.txt
/home/eml4u/EML4U/data/interim-results/20100408-indian-films.txt
/home/eml4u/EML4U/data/interim-results/20201101-indian-films.txt


In [4]:
# Load embeddings
def loadEmbeddings(file, note = "", printinfo = True):
    embeddings = numpy.loadtxt(file)
    if printinfo:
        print(str(embeddings.shape) + " " + note)
    return embeddings

embeddingsAmericanA = loadEmbeddings(fileEmbeddingsAmericanA, "AmericanA")
embeddingsAmericanB = loadEmbeddings(fileEmbeddingsAmericanB, "AmericanB")
embeddingsBritishA  = loadEmbeddings(fileEmbeddingsBritishA, "BritishA")
embeddingsBritishB  = loadEmbeddings(fileEmbeddingsBritishB, "BritishB")
embeddingsIndianA   = loadEmbeddings(fileEmbeddingsIndianA, "IndianA")
embeddingsIndianB   = loadEmbeddings(fileEmbeddingsIndianB, "IndianB")

# Create test embeddings (A,B: 2 points of time ; 3 entries/texts each ; 5 dimensions)
embeddingsTestA = numpy.array([[-2.2, -1.1, 0, 1.1, 2.2], [-2.2, -1.1, 0, 3.3, 4.4], [-2.2, -1.1, 0, 5.5, 6.6]])
embeddingsTestB = numpy.array([[-4.4, -2.2, 0, 1.1, 2.2], [-6.6, -3.3, 0, 1.1, 2.2], [-8.8, -4.4, 0, 1.1, 2.2]])
print(str(embeddingsTestA.shape) + " TestA")
print(str(embeddingsTestB.shape) + " TestB")

(1000, 768) AmericanA
(1000, 768) AmericanB
(2147, 768) BritishA
(2147, 768) BritishB
(3596, 768) IndianA
(3596, 768) IndianB
(3, 5) TestA
(3, 5) TestB


In [5]:
# Compute means
# https://numpy.org/doc/1.19/reference/generated/numpy.mean.html?highlight=mean#numpy.mean
# https://numpy.org/doc/1.19/reference/generated/numpy.reshape.html?highlight=reshape#numpy.reshape
def getMean(embeddings, note = "", printinfo = True):
    mean = numpy.mean(embeddings, axis=0)
    if printinfo:
        print(str(type(mean)) + " " + str(mean.shape) + " " +  note)
    return mean

# Compute means
meanAmericanA = getMean(embeddingsAmericanA, "AmericanA")
meanAmericanB = getMean(embeddingsAmericanB, "AmericanB")
meanBritishA  = getMean(embeddingsBritishA, "BritishA")
meanBritishB  = getMean(embeddingsBritishB, "BritishB")
meanIndianA   = getMean(embeddingsIndianA, "IndianA")
meanIndianB   = getMean(embeddingsIndianB, "IndianB")
meanTestA = getMean(embeddingsTestA, "TestA")
meanTestB = getMean(embeddingsTestB, "TestB")
print()

# Check means of test embeddings
if True:
    print("Testing computation of means")
    print(embeddingsTestA, "embeddingsTestA (input)")
    print(meanTestA, "meanTestA")
    print(embeddingsTestB, "embeddingsTestB (input)")
    print(meanTestB, "meanTestB")
    print()

<class 'numpy.ndarray'> (768,) AmericanA
<class 'numpy.ndarray'> (768,) AmericanB
<class 'numpy.ndarray'> (768,) BritishA
<class 'numpy.ndarray'> (768,) BritishB
<class 'numpy.ndarray'> (768,) IndianA
<class 'numpy.ndarray'> (768,) IndianB
<class 'numpy.ndarray'> (5,) TestA
<class 'numpy.ndarray'> (5,) TestB

Testing computation of means
[[-2.2 -1.1  0.   1.1  2.2]
 [-2.2 -1.1  0.   3.3  4.4]
 [-2.2 -1.1  0.   5.5  6.6]] embeddingsTestA (input)
[-2.2 -1.1  0.   3.3  4.4] meanTestA
[[-4.4 -2.2  0.   1.1  2.2]
 [-6.6 -3.3  0.   1.1  2.2]
 [-8.8 -4.4  0.   1.1  2.2]] embeddingsTestB (input)
[-6.6 -3.3  0.   1.1  2.2] meanTestB



In [6]:
# Cosine similarity
# https://scikit-learn.org/0.23/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html#sklearn.metrics.pairwise.cosine_similarity
# https://numpy.org/doc/1.19/reference/generated/numpy.reshape.html?highlight=reshape#numpy.reshape
def getPairwiseCosineSimilarity(a, b, note = "", printinfo = True):
    if printinfo:
        print(str(type(a)) + " " + str(a.shape) + "\n" + str(type(b)) + " " + str(b.shape))
    cosSim = sklearn.metrics.pairwise.cosine_similarity(a, b, dense_output=True)[0][0]
    if printinfo:
        print(str(cosSim) + " " + note)
    return cosSim
    
# https://scistatcalc.blogspot.com/2015/11/cosine-similarity-calculator.html
# -2.2, -1.1, 0, 3.3, 4.4
# -6.6, -3.3, 0, 1.1, 2.2
# Cosine Similarity between vectors 1 and 2 is 0.671317, cosine of 0.834812 radians

getPairwiseCosineSimilarity(meanTestA.reshape(1, -1), meanTestB.reshape(1, -1), "Test")

getPairwiseCosineSimilarity(meanAmericanA.reshape(1, -1), meanAmericanB.reshape(1, -1), "American")
getPairwiseCosineSimilarity(meanBritishA.reshape(1, -1), meanBritishB.reshape(1, -1), "British")
getPairwiseCosineSimilarity(meanIndianA.reshape(1, -1), meanIndianB.reshape(1, -1), "Indian")
print()

print(meanTestA.shape)
print(meanAmericanA.shape)

# TODO: These results seem to be wrong compared with computations in next cell.

# Note: Old calculated values were based on an incorrect mean.
# Returned value in cell 5: https://github.com/EML4U/EmbeddingsWikipedia/blob/f4a94871859a84cea31844b3f26ea34d3bd94791/cosine-similarity-wikipedia.ipynb
# Line: https://github.com/EML4U/EmbeddingsWikipedia/blob/f4a94871859a84cea31844b3f26ea34d3bd94791/cosine-similarity-wikipedia.ipynb?short_path=e52c59b#L138
#
# Old calculated values with reshape:
#0.9470747593129083 American
#0.9422465618828457 British
#0.931203987093214 Indian
# Old calculated values without reshape:
#0.952511701044914 American
#0.9618288510169046 British
#0.9760882993008537 Indian

<class 'numpy.ndarray'> (1, 5)
<class 'numpy.ndarray'> (1, 5)
0.6713171133426189 Test
<class 'numpy.ndarray'> (1, 768)
<class 'numpy.ndarray'> (1, 768)
0.9951524315697577 American
<class 'numpy.ndarray'> (1, 768)
<class 'numpy.ndarray'> (1, 768)
0.9936639191853995 British
<class 'numpy.ndarray'> (1, 768)
<class 'numpy.ndarray'> (1, 768)
0.9930823325071297 Indian

(5,)
(768,)


In [7]:
# Cosine similarity single entries
def getCosineSimilarity(a, b, indexA, indexB, note = "", printinfo = True):
    cosSim = sklearn.metrics.pairwise.cosine_similarity(a[indexA].reshape(1, -1), b[indexB].reshape(1, -1), dense_output=True)[0][0]
    if printinfo:
        print(str(cosSim) + " " + note)
    return cosSim

if True:
    getCosineSimilarity(embeddingsTestA, embeddingsTestB, 0, 0, "Test 0")
    
    getCosineSimilarity(embeddingsTestA, embeddingsTestB, 1, 1, "Test 1")
    # TODO: Value 0.67131711334261 is very similar to overall similarity in cell 6 -> error?
    
    getCosineSimilarity(embeddingsTestA, embeddingsTestB, 2, 2, "Test 2")
    print()
    
if True:
    getCosineSimilarity(embeddingsAmericanA, embeddingsAmericanB, 0, 0, "American 0")
    getCosineSimilarity(embeddingsAmericanA, embeddingsAmericanB, 1, 1, "American 1")
    getCosineSimilarity(embeddingsAmericanA, embeddingsAmericanB, 10, 10, "American 10")
    getCosineSimilarity(embeddingsAmericanA, embeddingsAmericanB, 100, 100, "American 100")
    getCosineSimilarity(embeddingsAmericanA, embeddingsAmericanB, 500, 500, "American 500")
    getCosineSimilarity(embeddingsAmericanA, embeddingsAmericanB, 600, 600, "American 600")
    print()
    getCosineSimilarity(embeddingsBritishA, embeddingsBritishB, 0, 0, "British 0")
    getCosineSimilarity(embeddingsBritishA, embeddingsBritishB, 1, 1, "British 1")
    getCosineSimilarity(embeddingsBritishA, embeddingsBritishB, 10, 10, "British 10")
    getCosineSimilarity(embeddingsBritishA, embeddingsBritishB, 100, 100, "British 100")
    getCosineSimilarity(embeddingsBritishA, embeddingsBritishB, 500, 500, "British 500")
    getCosineSimilarity(embeddingsBritishA, embeddingsBritishB, 600, 600, "British 600")
    print()
    getCosineSimilarity(embeddingsIndianA, embeddingsIndianB, 0, 0, "Indian 0")
    getCosineSimilarity(embeddingsIndianA, embeddingsIndianB, 1, 1, "Indian 1")
    getCosineSimilarity(embeddingsIndianA, embeddingsIndianB, 10, 10, "Indian 10")
    getCosineSimilarity(embeddingsIndianA, embeddingsIndianB, 100, 100, "Indian 100")
    getCosineSimilarity(embeddingsIndianA, embeddingsIndianB, 500, 500, "Indian 500")
    getCosineSimilarity(embeddingsIndianA, embeddingsIndianB, 600, 600, "Indian 600")
    print()

0.948683298050514 Test 0
0.671317113342619 Test 1
0.4939924295969613 Test 2

0.9525117010449139 American 0
0.9715444013842313 American 1
0.9724710753091028 American 10
0.9772824870224932 American 100
0.9393622073417478 American 500
0.9867714675621426 American 600

0.9618288510169046 British 0
0.9325135174538641 British 1
0.9473581782691437 British 10
0.9680476495788866 British 100
0.9310833202893275 British 500
0.8651356315148098 British 600

0.9760882993008546 Indian 0
0.9805102425307188 Indian 1
0.7447943708902642 Indian 10
0.6380358748619926 Indian 100
0.8617119348355946 Indian 500
0.8981347803389674 Indian 600

